In [ ]:
!pip install tensorflow

In [ ]:
# Paso 1: Cargar datos
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer # Changed import statement
from tensorflow.keras.utils import to_categorical # Changed import statement
from tensorflow.keras.preprocessing.sequence import pad_sequences # Changed import statement

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta = "/content/drive/MyDrive/Copies.xlsx"

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/Copies.xlsx")
df.head()

,Copie,Canal,Producto,Oferta de adquisición
0,Solicita tu tarjeta y reciban un bono exclusiv...,Referidos (MGM),Green,"$1,500 MXN + 10,000 puntos"
1,Ambos ganan: tú y quien te invitó. Solicita tu...,Referidos (MGM),Green,"$2,500 MXN"
2,Solicita tu tarjeta y reciban un bono exclusiv...,Referidos (MGM),Green,"$1,500 MXN + 10,000 puntos"
3,Recomienda y haz crecer tu comunidad Amex. Sol...,Referidos (MGM),Green,"$2,000 MXN"
4,"Invita, comparte y ganen juntos. Solicita tu G...",Referidos (MGM),Green,"20,000 puntos"


In [ ]:
# Función de limpieza
def limpiar_texto(copie):
    texto = copie.strip()
    texto = " ".join(texto.split())  # Eliminar espacios dobles
    texto = texto.replace(" m n", " M.N.")  # Uniformar "m n"
    texto = texto.replace(" puntos", " puntos")  # (opcional, en caso de errores)
    texto = texto.capitalize()  # Capitaliza primera letra
    if not texto.endswith("."):
        texto += "."
    return texto


In [ ]:
# Aplicar limpieza y eliminar duplicados
df["Copie"] = df["Copie"].apply(limpiar_texto)
df = df.drop_duplicates(subset=["Copie"])

In [ ]:
# Preparar lista de textos
texts = df["Copie"].tolist()

In [ ]:
# Paso 2: Tokenización por palabra
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
total_words = len(tokenizer.word_index) + 1

# Crear secuencias
input_sequences = []
for line in texts:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(2, len(token_list)+1):
        n_gram_sequence = token_list[:i]
        input_sequences.append(n_gram_sequence)

# Padding
data = pad_sequences(input_sequences, maxlen=max([len(x) for x in input_sequences]), padding='pre')

# Separar X e y
X = data[:, :-1]
y = data[:, -1]
y = to_categorical(y, num_classes=total_words)

In [ ]:
# Paso 3: Modelo
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=64, input_length=X.shape[1]))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Paso 4: Entrenamiento
model.fit(X, y, epochs=200, batch_size=128)

Epoch 1/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8548 - loss: 0.2807
Epoch 2/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8527 - loss: 0.2848
Epoch 3/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8545 - loss: 0.2836
Epoch 4/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8518 - loss: 0.2872
Epoch 5/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8563 - loss: 0.2819
Epoch 6/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8561 - loss: 0.2832
Epoch 7/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8522 - loss: 0.2861
Epoch 8/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8527 - loss: 0.2928
Epoch 9/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8502 - loss: 0.2877
Epoch 10/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8554 - loss: 0.2870
Epoch 11/200
131/131 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8550 - loss: 0.2834
Epoch 12/200
131/131 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Paso 5: Función para generar texto
import numpy as np

def generate_text_word(seed_text, next_words=20, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=X.shape[1], padding='pre')
        predictions = model.predict(token_list, verbose=0)[0]

        preds = np.log(predictions + 1e-8) / temperature
        exp_preds = np.exp(preds)
        predictions = exp_preds / np.sum(exp_preds)

        predicted_index = np.random.choice(range(len(predictions)), p=predictions)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [ ]:
# Ejemplo:
print(generate_text_word("Invita a tus amigos", next_words=15, temperature=0.7))

Invita a tus amigos beneficios con quien más confías solicita tu aeromexico gold y recibe 5 000 puntos otra


In [ ]:
# Lista de frases iniciales para probar
seeds = [
    "Solicita tu tarjeta",
    "Invita a tus amigos",
    "Activa tu tarjeta",
    "Solicita tu Aeromexico Gold",
    "Pide tu tarjeta adicional",
    "Haz un upgrade",
    "Obtén recompensas exclusivas"
]

def generar_variaciones(seeds, n_variantes=3, temperature=0.7):
    for seed in seeds:
        print(f"\n🔹 Seed: '{seed}'")
        for i in range(n_variantes):
            resultado = generate_text_word(seed, next_words=15, temperature=temperature)
            print(f"   {i+1}. {resultado}")

In [ ]:
generar_variaciones(seeds, n_variantes=5, temperature=0.3)


🔹 Seed: 'Solicita tu tarjeta'
   1. Solicita tu tarjeta ideal en pocos clics solicita tu platinum crédito y recibe 1 000 mxn lo 000
   2. Solicita tu tarjeta ideal en pocos clics solicita tu aeromexico green y recibe 10 000 puntos para tu
   3. Solicita tu tarjeta ideal en pocos clics solicita tu platinum y recibe 1 500 mxn 10 000 puntos
   4. Solicita tu tarjeta ideal en pocos clics solicita tu aeromexico blue y recibe 10 000 puntos para tu
   5. Solicita tu tarjeta y reciban un bono exclusivo los dos solicita tu aeromexico green y recibe 20 000

🔹 Seed: 'Invita a tus amigos'
   1. Invita a tus amigos beneficios con quien más confías solicita tu aeromexico gold y recibe 10 000 puntos ahora
   2. Invita a tus amigos beneficios con quien más confías solicita tu aeromexico gold y recibe 10 000 puntos que
   3. Invita a tus amigos beneficios con quien más confías solicita tu aeromexico gold y recibe 1 500 mxn 10
   4. Invita a tus amigos beneficios con quien más confías solicita tu aerom

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
# Opciones para la demo
canales = [
    "Referidos (MGM)", "Google", "Social Media", "Partner (Aeroméxico)",
    "Afiliados", "Suplementarias", "Upgrades", "Companion",
    "Reverse Companion", "Xsell"
]

In [ ]:
ofertas = [
    "$1,000 MXN", "$1,500 MXN", "$2,000 MXN",
    "5,000 puntos", "10,000 puntos", "20,000 puntos"
]

In [ ]:
productos_clave = ["green", "gold", "platinum", "aeromexico", "crédito"]
recompensas_clave = [o.lower() for o in ofertas]

In [ ]:
# Validación automática
def es_valido(copy):
    contiene_producto = any(p in copy.lower() for p in productos_clave)
    contiene_recompensa = any(r in copy.lower() for r in recompensas_clave)
    return contiene_producto and contiene_recompensa

In [ ]:
# UI widgets
canal_dropdown = widgets.Dropdown(options=canales, description="Canal:")
oferta_dropdown = widgets.Dropdown(options=ofertas, description="Oferta:")
seed_input = widgets.Text(value="Solicita tu tarjeta", description="Seed:")
temperature_slider = widgets.FloatSlider(value=0.7, min=0.2, max=1.2, step=0.1, description="Temperatura")
output_area = widgets.Output()

def generar_copy_interactivo(b):
    output_area.clear_output()
    seed = seed_input.value
    temp = temperature_slider.value
    canal = canal_dropdown.value
    oferta = oferta_dropdown.value

    resultado = generate_text_word(seed, next_words=15, temperature=temp)

    with output_area:
        print(f"📝 Copy generado:\n{resultado}")
        if es_valido(resultado):
            print(f"\n✅ Copy válido: contiene producto + recompensa.")
            print(f"🎯 Canal: {canal} | Oferta esperada: {oferta}")
        else:
            print(f"\n⚠️ El copy generado no cumple con los criterios.")

In [ ]:
# Botón de ejecución
generar_button = widgets.Button(description="Generar Copy")
generar_button.on_click(generar_copy_interactivo)

In [ ]:
# Mostrar interfaz
display(canal_dropdown, oferta_dropdown, seed_input, temperature_slider, generar_button, output_area)

Dropdown(description='Canal:', options=('Referidos (MGM)', 'Google', 'Social Media', 'Partner (Aeroméxico)', '…

Dropdown(description='Oferta:', options=('$1,000 MXN', '$1,500 MXN', '$2,000 MXN', '5,000 puntos', '10,000 pun…

Text(value='Solicita tu tarjeta', description='Seed:')

FloatSlider(value=0.7, description='Temperatura', max=1.2, min=0.2)

Button(description='Generar Copy', style=ButtonStyle())

Output()